In [264]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/energy-dataset/energydata_complete.csv


In [265]:
# First we import the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
scaler = MinMaxScaler()
linear_models = LinearRegression()

In [266]:
df = pd.read_csv("/kaggle/input/energy-dataset/energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [267]:
df.shape

(19735, 29)

In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [269]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [270]:
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [271]:
df.columns


Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [272]:
df1 =df [["T2", "T6"]]

In [273]:
df1.describe()

,T2,T6
count,19735.000000,19735.000000
mean,20.341219,7.910939
std,2.192974,6.090347
min,16.100000,-6.065000
25%,18.790000,3.626667
50%,20.000000,7.300000
75%,21.500000,11.256000
max,29.856667,28.290000


In [274]:
df1.head()

,T2,T6
0,19.2,7.026667
1,19.2,6.833333
2,19.2,6.560000
3,19.2,6.433333
4,19.2,6.366667


In [275]:


# Extract the input (x) and output (y) variables
x = df['T2'].values.reshape(-1, 1)  # Reshape to a column vector
y = df['T6'].values

# Create a LinearRegression model and fit the data
model = LinearRegression()
model.fit(x, y)

# Get the R^2 value of the model
r_squared = model.score(x, y)

# Print the R^2 value
print("R^2 value:", round(r_squared, 2))


R^2 value: 0.64


In [276]:
# Remove columns "date" and "lights"
columns_to_remove = ["date", "lights"]
data = df.drop(columns=columns_to_remove)

In [277]:
# import the scaler from library
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
# df_copy_scaled

# separating the features and targets
features = df_scaled.drop(columns=['Appliances'], axis=1)
target = df_scaled.Appliances

In [278]:
# splitting  to train and test sets
X1_train, X1_test, y1_train, y1_test = train_test_split(features, target, test_size=0.3, random_state=42)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((13814, 26), (5921, 26), (13814,), (5921,))

In [279]:
# importing the libraries for model building
from sklearn.linear_model import LinearRegression
model_mul= LinearRegression()
model_mul.fit(X1_train, y1_train)

# lets predict
predictions_mul = model_mul.predict(X1_test)

# Model Evaluation
# model_mul_score = model_mul.score(y_test, predictions_mul)
# model_mul_score

mae_mul = round(mean_absolute_error(y1_test, predictions_mul), 2)
print(f'The Mean Absolute Error in 2 decimal places is {mae_mul}')

The Mean Absolute Error in 2 decimal places is 0.05


In [280]:

residual = round(np.sum(np.square(Y_test - predicted_values)), 2)
print(f'The Residual in 2 decimal places is {residual}')

The Residual in 2 decimal places is 45.35


In [281]:
rmse = round(np.sqrt(np.mean(np.square(y1_test - predictions_mul))), 3)
print(f'The Root Mean Squared Error is {rmse}')

The Root Mean Squared Error is 0.088


In [282]:
# coefficient of determination is r2 score
cod = round(r2_score(y1_test, predictions_mul), 2)
print(f'The Coefficient of Determination (r2 Score) is {cod}')

The Coefficient of Determination (r2 Score) is 0.15


In [283]:
weight = pd.Series(model_mul.coef_, features.columns).sort_values()
# weight
weight_df = pd.DataFrame(weight).reset_index()
# weight_df.head()

weight_df.columns = ['Features', 'Weight']
weight_df
print(f'The features with the lowest and highest weights respectively are RH_2, RH_1')

The features with the lowest and highest weights respectively are RH_2, RH_1


In [284]:

ridge_model = Ridge(alpha=0.4)
ridge_model.fit(X1_train, y1_train)
predictions_ridge = ridge_model.predict(X1_test)

rmse_ridge = round(np.sqrt(mean_squared_error(y1_test, predictions_ridge)), 3)
expected_rmse = 0.088

if rmse_ridge == expected_rmse:
    print(f'RMSE for Linear Regression is equal to the RMSE for Ridge = {expected_rmse}')
else:
    print(f'RMSE for Linear Regression is not equal to the RMSE for Ridge = {expected_rmse}')


RMSE for Linear Regression is equal to the RMSE for Ridge = 0.088


In [285]:
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X1_train, y1_train)
predictions_lasso = lasso_model.predict(X1_test)

weights_lasso = pd.Series(lasso_model.coef_, features.columns).sort_values()
weights_lasso_df = pd.DataFrame(weights_lasso).reset_index()
weights_lasso_df.columns = ['Features', 'Weights']

weights_lasso_df


,Features,Weights
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [286]:
non_zero_feature_weights = weights_lasso_df[weights_lasso_df.Weights!=0]
non_zero_feature_weights
print(f'There are 4 non-zero feature weights')

There are 4 non-zero feature weights


In [287]:
rmse_lasso = round(np.sqrt(mean_squared_error(y1_test, predictions_lasso)), 3)
print(f'The RMSE with the Lasso Regression is {rmse_lasso}')


The RMSE with the Lasso Regression is 0.094
